In [2]:
import glob
import shutil 
from netCDF4 import Dataset
import numpy as np
import os
import yaml 

from obspy.core import Stream, Trace, UTCDateTime, Stats
from obspy.core.event import read_events


In [5]:
# read event location
# with open('../Runs/CartesianModel0007_random0000/input/inparam.output.yaml', 'r') as file:
    output_yaml = yaml.load(file, Loader=yaml.FullLoader)

In [11]:
# output_yaml['list_of_station_groups'][0]['Synthetic_Stations']['temporal']['time_window']


[0, 1.5]

In [16]:
# data dir
# ModelNameList = ['CartesianModel0007_5pct','CartesianModel0007_10pct','CartesianModel0007_20pct']
ModelNameList = []
for imodel in np.arange(0,21):
    ModeName = 'CartesianModel0007_random%04d' %imodel
    ModelNameList.append(ModeName)

# wave dimension to animation
output_channel = 'X'
# wave_dim_s = output_channel.index('s')
# wave_dim_p = output_channel.index('p')
# wave_dim_z = output_channel.index('z')
wave_dim_X = output_channel.index('X')

for ModelName in ModelNameList:
    # data_dir = '../Runs/%s/output/elements/orthogonal_azimuthal_slices' %ModelName
    RunPath = '../Runs/%s' %ModelName

    NETCDFDir = '../DataSet/%s' %ModelName
    if not os.path.exists(NETCDFDir):
        os.mkdir(NETCDFDir)
    StationInfo = np.loadtxt(RunPath+'/input/Synthetic_Stations_Cartesian.txt',dtype=str, skiprows=3)
    stalatlon_dict = {}
    for item in StationInfo:
        stkey = item[1]+'.'+item[0]
        x = float(item[2])
        y = float(item[3])
        if stkey not in stalatlon_dict.keys():
            stalatlon_dict[stkey] = []
        stalatlon_dict[stkey].append((x, y))


    StartTime, EndTime = 0, 1.5
    resample_rate = 0.05
    lowpass_freq = 20

    PointPerTrace = int((EndTime - StartTime)/resample_rate)+1

    istation = 0

    if os.path.exists(NETCDFDir+'/seismogram.nc'):
        os.remove(NETCDFDir+'/seismogram.nc')

    nc = Dataset(NETCDFDir+'/seismogram.nc', 'w')
    nc.createDimension('npoint', size=PointPerTrace*len(stalatlon_dict))
    nc.createVariable('x', float, dimensions=('npoint'))
    nc.createVariable('y', float, dimensions=('npoint'))
    nc.createVariable('z', float, dimensions=('npoint'))
    nc.createVariable('time', float, dimensions=('npoint'))
    nc.createVariable('X', float, dimensions=('npoint'))


    GSNDir = RunPath + '/output/stations/Synthetic_Stations'

    # read rank-station info
    rank_station_info = np.loadtxt(GSNDir + '/rank_station.info', dtype=str, skiprows=1)

    # dict: mpi-rank -> [station keys]
    rank_station_dict = {}
    for item in rank_station_info:
        rank = item[0]
        stkey = item[1]
        inrank_index = item[2]
        # initialize with an empty array if rank does not exists in rank_station_dict
        if rank not in rank_station_dict.keys():
            rank_station_dict[rank] = []
        # append the station
        rank_station_dict[rank].append([stkey, inrank_index])
        
    # loop over mpi-ranks to read data
    for rank in rank_station_dict.keys():
        f = Dataset(GSNDir + '/axisem3d_synthetics.nc.rank%s' %rank, 'r')
        time = f.variables['data_time'][:]

        for [StationName, inrank_index] in rank_station_dict[rank]:
            stax = stalatlon_dict[StationName][0][0]
            stay = stalatlon_dict[StationName][0][1]
            staz = 0

            # trace header
            stats = Stats()
            stats.starttime = UTCDateTime(time[0])
            stats.delta = UTCDateTime(time[1] - time[0])
            stats.npts = len(time)

            # stream
            stream = Stream()
            for ich, ch in enumerate(output_channel):
                stats.channel = ch  
                # default unit is km
                stream.append(Trace(f.variables['data_wave'][int(inrank_index)][ich], header=stats))


            stream.filter('lowpass', freq=lowpass_freq)
            stream.resample(resample_rate)
            # stream = stream.slice(UTCDateTime(int(arrivals[0].time)+StartTime), UTCDateTime(int(arrivals[0].time)+EndTime))
            stream = stream.slice(UTCDateTime(StartTime), UTCDateTime(EndTime))
            npoint_persta = len(stream[0].data)
        
            nc['x'][istation*npoint_persta:(istation+1)*npoint_persta] = stax
            nc['y'][istation*npoint_persta:(istation+1)*npoint_persta] = stay
            nc['z'][istation*npoint_persta:(istation+1)*npoint_persta] = staz

            # nc['time'][istation*npoint_persta:(istation+1)*npoint_persta] = stream[0].times() + int(arrivals[0].time) + StartTime
            nc['time'][istation*npoint_persta:(istation+1)*npoint_persta] = stream[0].times()

            nc['X'][istation*npoint_persta:(istation+1)*npoint_persta] = stream[0].data

            istation += 1

    nc.close()

    print(NETCDFDir+'/seismogram.nc', " set up and saved!")

../DataSet/CartesianModel0007_random0000/seismogram.nc  set up and saved!
../DataSet/CartesianModel0007_random0001/seismogram.nc  set up and saved!
../DataSet/CartesianModel0007_random0002/seismogram.nc  set up and saved!
../DataSet/CartesianModel0007_random0003/seismogram.nc  set up and saved!
../DataSet/CartesianModel0007_random0004/seismogram.nc  set up and saved!
../DataSet/CartesianModel0007_random0005/seismogram.nc  set up and saved!
../DataSet/CartesianModel0007_random0006/seismogram.nc  set up and saved!
../DataSet/CartesianModel0007_random0007/seismogram.nc  set up and saved!
../DataSet/CartesianModel0007_random0008/seismogram.nc  set up and saved!
../DataSet/CartesianModel0007_random0009/seismogram.nc  set up and saved!
../DataSet/CartesianModel0007_random0010/seismogram.nc  set up and saved!
../DataSet/CartesianModel0007_random0011/seismogram.nc  set up and saved!
../DataSet/CartesianModel0007_random0012/seismogram.nc  set up and saved!
../DataSet/CartesianModel0007_random00

In [15]:
npoint_persta
PointPerTrace

31

In [21]:
import shutil
# Move netcdf file
# shutil.copy(RunPath+'/input/box_tomography.nc',NETCDFDir)
shutil.copy(RunPath+'/tools/UnifromEarthCore.bm',NETCDFDir)

'/raid1/TomoNet/Runs/Sample/model0011/UnifromEarthCore.bm'